<a href="https://colab.research.google.com/github/drwbkr1/Grad504-K-Nearest-Neighbor-Classifier/blob/main/KNN_Project_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#load dataset
import pandas as pd
import numpy as np

file= ("/content/drive/MyDrive/Colab Notebooks/datasets/Prog1data.csv")


In [19]:
#Any dataset is verified for basic parameters before preprocessing.
def load_csv(file):
  df= pd.read_csv(file)

  #Check for data in dataset, numerical features for supporting floating-point inputs
  if df.shape[1] < 2:
    raise ValueError("File must have at least one feature and a target")
  try:
    X_df= df.iloc[:, :-1].astype(float)
  except ValueError:
    raise ValueError("Error: Features must be numerical")

  #Split dataset into features and target, features to floating point.
  X= df.iloc[:, :-1].to_numpy(dtype=float)
  y= df.iloc[:, -1].to_numpy()

  #Check for feature-target congruency, feature congruency
  if not X.shape[0]==y.shape[0]:
    raise ValueError("Error: Number of features and target must be the same")
  if not all(len(features)==len(X[0]) for features in X):
    raise ValueError("Error: All features must have the same length")

  return X, y

#Load data into core variables
X, y= load_csv(file)
for i in range(3):
  print(X[i], y[i])
print(X.dtype, y.dtype)

[ 8.  9.  0. 26. 10. 25.  7. 68.  2.  7. 10.] +
[ 8.  8.  0. 23.  9. 15.  5. 70.  3.  7. 10.] +
[11.  3.  6. 19.  8. 17.  6. 80.  2.  6. 10.] +
float64 object
